In [142]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import pandas as pd
import plotly as plt
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
import string
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
import re
from nltk.probability import FreqDist
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import numpy as np
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

In [101]:
PATH_COMMON = "C:\\Users\\Админ\\Desktop\\pandas\\"
PATH_TRAIN = PATH_COMMON + "train.csv"

In [102]:
data = pd.read_csv(PATH_TRAIN, sep='\t', encoding='cp1252')

In [103]:
columns = list(data.columns)[0].split(",")
columns

['Id', 'Hotel_name', 'Review_Title', 'Review_Text', 'Rating']

In [ ]:
# В левом столбце количество элементов списка после сплитка исходного объекта
# В правом столбце количество таких объектов

#1 - 99
#2 - 122
#3 - 30
#4 - 92
#5 - 1160
#6 - 485
#7 - 287
#8 - 145
#9 - 87
#10 - 49

# после анализа можно смело выкидвать объекты, после сплита которых остаётся: 1, 2, 3, 4. В этих объектах 
# сильно преимущественно не хватает каких - то элементов для анализа. Также преимущественно не хватает
# рейтингов.
# итого мы теряем 206 грязных объектов
# самые чистые и удобные объекты это 5
# остальные требуют более качественный split

In [104]:
idents = []
names = []
titles = []
texts = []
ratings = []

objects = list(data["Id,Hotel_name,Review_Title,Review_Text,Rating"])

for i in objects:
    if len(i.split(",")) <= 4:
        objects.remove(i)

        
nums = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "."]        
for i in objects:
    words = i.strip().split(",")
    
    if len(words) == 5:
        num_in_text = False
        text_in_nums = False
        
        ident, name, title, text, rating = words
        
        for j in text:
            if j in nums[0:10]:
                num_in_text = True
                break
        
        for k in rating:
            if k not in nums:
                text_in_nums = True
                break

        if not num_in_text and not text_in_nums:
            idents.append(ident)
            names.append(name.lower())
            titles.append(title.lower())
            texts.append(text.lower())
            ratings.append(rating)

        objects.remove(i)

In [127]:
names = np.array(names)
titles = np.array(titles)
texts = np.array(texts)
ratings = np.array(ratings)

text_final = []
for i in range(texts.shape[0]):
    text_final.append(names[i] + " " + titles[i] + " " + texts[i])

text_final = np.array(text_final)    
    
columns = ["text", "rating"]

data = pd.DataFrame(columns = columns)
data["text"] = text_final
data["rating"] = ratings
data["rating"] = data["rating"].apply(lambda x: int(float(x)))

In [128]:
data

,text,rating
0,the royal regency no worth the rating shown in...,71
1,metro grand lovely hotel the hotel is pretty c...,80
2,the park chennai hotel staff are not co-ordin...,40
3,treebo j's five two boutique good appearance &...,86
4,bhimaas great hotel the hotel is great and its...,86
...,...,...
635,park hyatt super hotal nice place good staff i...,65
636,lemon tree chennai very poor ac and service ve...,65
637,itc grand chola best hotel in india !! the mom...,100
638,hilton chennai five star service lovely hotel ...,100


In [129]:
train_df, test_df = train_test_split(data, test_size=0.1, shuffle=True)

In [130]:
snowball = SnowballStemmer(language="english")
stop_words = stopwords.words("english") + ["of", "and", "..", "...", "....", ".....", "'s"]
spec_chars = string.punctuation

def get_tokens(sentence):
    tokens = word_tokenize(sentence, language="english")
    
    tokens = [i for i in tokens if i not in spec_chars]
    tokens = [i for i in tokens if i not in stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    
    return tokens

In [131]:
prepared = []
for i in range(data_train.shape[0]):
    prep = get_tokens(data["text"][i])
    print(prep)
    for j in prep:
        prepared.append(j)
        
FreqDist(prepared).most_common(300)

['royal', 'regenc', 'worth', 'rate', 'shown', 'websit', 'price', 'ok', 'worth', 'rate', 'shown', 'servic', 'good', 'room', 'well', 'maintain', 'room', 'spacious', 'illumin', 'room', 'poor', 'tv', 'remot', 'work', 'carri', 'good', 'mosquito', 'repal', 'locat', 'wise', 'excel', 'want', 'stay', 'near', 'central', 'chennai']
['metro', 'grand', 'love', 'hotel', 'hotel', 'pretti', 'clean', 'excel', 'bed', 'clean', 'bath', 'wifi', 'work', 'probabl', 'one', 'area', 'need', 'improv', 'busi', 'center', 'got', 'printout', 'buffet', 'breakfast', 'south', 'indian', 'toast', 'bread']
['park', 'chennai', 'hotel', 'staff', 'co-ordin', 'well', 'room', 'servic', 'bill', 'payment']
['treebo', 'j', 'five', 'two', 'boutiqu', 'good', 'appear', 'amaz', 'hotel', 'stay', 'amaz', 'got', 'mani', 'benefit', 'handl', 'us', 'smooth', 'hotel', 'appear', 'good', 'member', 'like', 'good', 'particip', 'method', 'serv', 'differ', 'indian', 'hotel', 'think', 'give', 'complet', 'approach', 'us']
['bhimaa', 'great', 'hotel

['peac', 'inn', 'good', 'facil', 'clean', 'room', 'overal', 'good', 'stay', 'facil', 'room', 'good', 'peopl', 'cooper', 'facil', 'mention', 'time', 'book', 'said', 'provid', 'room', 'clean', 'breakfast', 'room', 'servic', 'good']
['fabhotel', 'metro', 'manor', 'central', 'station', 'nice', 'comfort', 'well', 'afford', 'good', 'park', 'facil', 'decent', 'buffet', 'hall', 'neat', 'clean', 'even', 'sea', 'breez', 'good', 'ambianc', 'recept', 'room', 'servic', 'beimprov']
['itc', 'grand', 'chola', 'excel', 'stay', 'valu', 'money', 'valu', 'money', 'staff', 'friend', 'help', 'breakfast', 'serv', 'realli', 'excel', 'staff', 'front', 'desk', 'help', 'room', 'clean', 'well', 'maintain', 'excel', 'lobbi', 'wonder', 'stay', 'hotel', 'locat', 'excel']
['brownstar', 'boutiqu', 'hotel', 'tasti', 'food', 'activ', 'kid', 'went', 'chennai', 'attend', 'birthday', 'function', 'friend', 'kid', 'hotel', 'nice', 'arrang', 'complet', 'own', 'hotel', 'made', 'food', 'tasti', 'also', 'sever', 'fun', 'activ', 

['e', 'hotel', 'nice', 'hotel', 'stay', 'shop', 'area', 'hotel', 'interior', 'realli', 'modern', 'spacious', 'strateg', 'locat', 'ad', 'valu', 'money', 'hotel', 'staff', 'friend', 'help', 'fact', 'even', 'help', 'us', 'get', 'movi', 'ticket', 'rush', 'busi', 'schedul', 'best', 'hotel', 'attach', 'new', 'shop', 'mall', 'get', 'a-z', "n't", 'need', 'spend', 'money', 'transport', 'worri', 'shop', 'till', 'late', 'night', 'definit', 'visit', 'hotel']
['lotel', 'hotel', 'breakfast', 'could', 'better']
['palac', 'lodg', 'overal', 'worst', 'experi', 'book', 'lodg', 'visit', 'lodg', 'stay', 'told', "n't", 'receiv', 'payment', 'makemytrip', 'paid', 'money', 'room', 'second', 'time', 'alreadi', 'paid', 'money', 'onlin', 'time', 'book', 'hotel', 'mainten', 'also', 'worst']
['itc', 'grand', 'chola', 'fantast', 'hotel', 'opul', 'best', 'itc', 'grand', 'chola', 'awesom', 'hotel', 'great', 'facil', 'friend', 'help', 'staff']
['raj', 'park', 'chennai', 'excel', 'stay', 'overal', 'stay', 'good']
['radh

KeyError: 640

In [132]:
model_pipeline = Pipeline([ 
    ("vectorizer", TfidfVectorizer(tokenizer=lambda x: get_tokens(x))), 
    ("model", CatBoostRegressor()) 
]) 

In [134]:
model_pipeline.fit(train_df["text"], train_df["rating"])

Learning rate set to 0.037525
0:	learn: 20.0613561	total: 164ms	remaining: 2m 43s
1:	learn: 19.9864404	total: 170ms	remaining: 1m 24s
2:	learn: 19.8980562	total: 175ms	remaining: 58.1s
3:	learn: 19.8041891	total: 181ms	remaining: 45s
4:	learn: 19.7054957	total: 186ms	remaining: 37.1s
5:	learn: 19.6265890	total: 192ms	remaining: 31.8s
6:	learn: 19.5343495	total: 198ms	remaining: 28.1s
7:	learn: 19.4606159	total: 203ms	remaining: 25.2s
8:	learn: 19.3841983	total: 209ms	remaining: 23s
9:	learn: 19.3080992	total: 214ms	remaining: 21.2s
10:	learn: 19.2106269	total: 220ms	remaining: 19.8s
11:	learn: 19.1302249	total: 225ms	remaining: 18.5s
12:	learn: 19.0788669	total: 231ms	remaining: 17.5s
13:	learn: 19.0308393	total: 236ms	remaining: 16.6s
14:	learn: 18.9469084	total: 242ms	remaining: 15.9s
15:	learn: 18.8675628	total: 247ms	remaining: 15.2s
16:	learn: 18.7896834	total: 253ms	remaining: 14.6s
17:	learn: 18.7271603	total: 258ms	remaining: 14.1s
18:	learn: 18.6603509	total: 264ms	remaining: 

166:	learn: 14.0836232	total: 1.09s	remaining: 5.44s
167:	learn: 14.0566958	total: 1.1s	remaining: 5.43s
168:	learn: 14.0331765	total: 1.1s	remaining: 5.42s
169:	learn: 14.0058640	total: 1.11s	remaining: 5.41s
170:	learn: 13.9863512	total: 1.11s	remaining: 5.4s
171:	learn: 13.9659341	total: 1.12s	remaining: 5.39s
172:	learn: 13.9424842	total: 1.12s	remaining: 5.38s
173:	learn: 13.9306824	total: 1.13s	remaining: 5.37s
174:	learn: 13.9202839	total: 1.14s	remaining: 5.35s
175:	learn: 13.9102815	total: 1.14s	remaining: 5.34s
176:	learn: 13.8839514	total: 1.15s	remaining: 5.33s
177:	learn: 13.8602770	total: 1.15s	remaining: 5.32s
178:	learn: 13.8478490	total: 1.16s	remaining: 5.31s
179:	learn: 13.8252791	total: 1.16s	remaining: 5.29s
180:	learn: 13.8163191	total: 1.17s	remaining: 5.28s
181:	learn: 13.7952628	total: 1.17s	remaining: 5.27s
182:	learn: 13.7735862	total: 1.18s	remaining: 5.27s
183:	learn: 13.7508086	total: 1.19s	remaining: 5.26s
184:	learn: 13.7190791	total: 1.19s	remaining: 5.

330:	learn: 10.9657415	total: 2s	remaining: 4.04s
331:	learn: 10.9593836	total: 2s	remaining: 4.03s
332:	learn: 10.9439601	total: 2.01s	remaining: 4.03s
333:	learn: 10.9316888	total: 2.02s	remaining: 4.02s
334:	learn: 10.9112463	total: 2.02s	remaining: 4.01s
335:	learn: 10.9049642	total: 2.03s	remaining: 4.01s
336:	learn: 10.8927440	total: 2.03s	remaining: 4s
337:	learn: 10.8867103	total: 2.04s	remaining: 3.99s
338:	learn: 10.8770316	total: 2.04s	remaining: 3.99s
339:	learn: 10.8646695	total: 2.05s	remaining: 3.98s
340:	learn: 10.8584061	total: 2.06s	remaining: 3.97s
341:	learn: 10.8396841	total: 2.06s	remaining: 3.97s
342:	learn: 10.8209047	total: 2.07s	remaining: 3.96s
343:	learn: 10.8102233	total: 2.07s	remaining: 3.95s
344:	learn: 10.7945473	total: 2.08s	remaining: 3.95s
345:	learn: 10.7804136	total: 2.08s	remaining: 3.94s
346:	learn: 10.7664912	total: 2.09s	remaining: 3.93s
347:	learn: 10.7574196	total: 2.1s	remaining: 3.93s
348:	learn: 10.7428691	total: 2.1s	remaining: 3.92s
349:

494:	learn: 9.0109229	total: 2.91s	remaining: 2.97s
495:	learn: 8.9982168	total: 2.92s	remaining: 2.97s
496:	learn: 8.9904313	total: 2.92s	remaining: 2.96s
497:	learn: 8.9853697	total: 2.93s	remaining: 2.95s
498:	learn: 8.9803612	total: 2.94s	remaining: 2.95s
499:	learn: 8.9682469	total: 2.94s	remaining: 2.94s
500:	learn: 8.9569860	total: 2.95s	remaining: 2.94s
501:	learn: 8.9491609	total: 2.95s	remaining: 2.93s
502:	learn: 8.9442216	total: 2.96s	remaining: 2.92s
503:	learn: 8.9394881	total: 2.96s	remaining: 2.92s
504:	learn: 8.9202187	total: 2.97s	remaining: 2.91s
505:	learn: 8.9064540	total: 2.97s	remaining: 2.9s
506:	learn: 8.8985532	total: 2.98s	remaining: 2.9s
507:	learn: 8.8913265	total: 2.98s	remaining: 2.89s
508:	learn: 8.8867403	total: 2.99s	remaining: 2.88s
509:	learn: 8.8819330	total: 3s	remaining: 2.88s
510:	learn: 8.8637171	total: 3s	remaining: 2.87s
511:	learn: 8.8554445	total: 3.01s	remaining: 2.87s
512:	learn: 8.8442658	total: 3.01s	remaining: 2.86s
513:	learn: 8.832562

660:	learn: 7.6155592	total: 3.82s	remaining: 1.96s
661:	learn: 7.6116673	total: 3.83s	remaining: 1.96s
662:	learn: 7.6059285	total: 3.83s	remaining: 1.95s
663:	learn: 7.5993682	total: 3.84s	remaining: 1.94s
664:	learn: 7.5955217	total: 3.85s	remaining: 1.94s
665:	learn: 7.5846307	total: 3.85s	remaining: 1.93s
666:	learn: 7.5762916	total: 3.86s	remaining: 1.93s
667:	learn: 7.5724766	total: 3.86s	remaining: 1.92s
668:	learn: 7.5669821	total: 3.87s	remaining: 1.91s
669:	learn: 7.5596758	total: 3.87s	remaining: 1.91s
670:	learn: 7.5534818	total: 3.88s	remaining: 1.9s
671:	learn: 7.5487546	total: 3.88s	remaining: 1.9s
672:	learn: 7.5350900	total: 3.89s	remaining: 1.89s
673:	learn: 7.5245009	total: 3.9s	remaining: 1.88s
674:	learn: 7.5150328	total: 3.9s	remaining: 1.88s
675:	learn: 7.5112189	total: 3.91s	remaining: 1.87s
676:	learn: 7.5034747	total: 3.91s	remaining: 1.87s
677:	learn: 7.4995285	total: 3.92s	remaining: 1.86s
678:	learn: 7.4936356	total: 3.92s	remaining: 1.85s
679:	learn: 7.48

825:	learn: 6.5250240	total: 4.73s	remaining: 997ms
826:	learn: 6.5209635	total: 4.74s	remaining: 991ms
827:	learn: 6.5178593	total: 4.74s	remaining: 985ms
828:	learn: 6.5100611	total: 4.75s	remaining: 980ms
829:	learn: 6.5014455	total: 4.75s	remaining: 974ms
830:	learn: 6.4943314	total: 4.76s	remaining: 968ms
831:	learn: 6.4841538	total: 4.76s	remaining: 962ms
832:	learn: 6.4774777	total: 4.77s	remaining: 956ms
833:	learn: 6.4744055	total: 4.78s	remaining: 951ms
834:	learn: 6.4641339	total: 4.78s	remaining: 945ms
835:	learn: 6.4595975	total: 4.79s	remaining: 939ms
836:	learn: 6.4543802	total: 4.79s	remaining: 933ms
837:	learn: 6.4512903	total: 4.8s	remaining: 928ms
838:	learn: 6.4482259	total: 4.8s	remaining: 922ms
839:	learn: 6.4451963	total: 4.81s	remaining: 916ms
840:	learn: 6.4369045	total: 4.82s	remaining: 910ms
841:	learn: 6.4290659	total: 4.82s	remaining: 905ms
842:	learn: 6.4164938	total: 4.83s	remaining: 899ms
843:	learn: 6.4127552	total: 4.83s	remaining: 893ms
844:	learn: 6.

991:	learn: 5.6316220	total: 5.64s	remaining: 45.5ms
992:	learn: 5.6291090	total: 5.65s	remaining: 39.8ms
993:	learn: 5.6244986	total: 5.65s	remaining: 34.1ms
994:	learn: 5.6204022	total: 5.66s	remaining: 28.4ms
995:	learn: 5.6178772	total: 5.67s	remaining: 22.8ms
996:	learn: 5.6113475	total: 5.67s	remaining: 17.1ms
997:	learn: 5.6024211	total: 5.68s	remaining: 11.4ms
998:	learn: 5.5998796	total: 5.68s	remaining: 5.69ms
999:	learn: 5.5906010	total: 5.69s	remaining: 0us


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x000002F04E563760>)),
                ('model',
                 <catboost.core.CatBoostRegressor object at 0x000002F04CBA9A80>)])

In [145]:
print("MSE: ", mean_squared_error(test_df["rating"], model_pipeline.predict(test_df["text"])))
print("RMSE: ", mean_squared_error(test_df["rating"], model_pipeline.predict(test_df["text"]), squared=False))
print("MAPE: ", mean_absolute_percentage_error(test_df["rating"], model_pipeline.predict(test_df["text"])) * 100)

MSE:  223.2390494123682
RMSE:  14.94118634554727
MAPE:  20.35703699874859
